### Graph

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
sc =SparkContext()
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf, lit
from pyspark.sql.types import IntegerType
from pyspark import SQLContext
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import Normalizer
from pyspark.sql import SparkSession

sqlContext = SQLContext(sc)
spark = SparkSession \
    .builder \
    .appName("graph") \
    .getOrCreate()

In [2]:
# dataframe1 = sqlContext.read.json('hdfs://localhost:1234/user/tl2861/hw3/train.json')
df3 = sqlContext.read.json('../../data/AA/wiki_*')
df3.printSchema()

root
 |-- id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)



In [ ]:
# df3.createOrReplaceTempView("wikinews")
# sqlDF3 = spark.sql("select count(*) from wikinews")
# sqlDF3.show()

In [28]:
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="[^A-Za-z]+", toLowercase=True)
tokenizedData = regexTokenizer.transform(df3)
stopWordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
filteredData = stopWordsRemover.transform(tokenizedData)
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=20)
featurizedData = hashingTF.transform(filteredData)
idf= IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(featurizedData)
data1 = idfModel.transform(featurizedData)
normalizer = Normalizer(inputCol="features", outputCol="norm")
data = normalizer.transform(data1)
data = data.sample(False, 0.2, 0)

In [29]:
# data1.show()
data.count()

4381

In [30]:
import numpy as np
feature = np.array(data.select('norm').collect())
# delete the 1 size axis
feature = np.squeeze(feature)
print(feature)

[[0.29313942 0.12286993 0.11253153 ... 0.30024093 0.30916789 0.29102504]
 [0.25695014 0.15955723 0.2045847  ... 0.2079407  0.24088874 0.12957297]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.15396869 0.24585243 0.15761632 ... 0.12015139 0.18558571 0.12478215]
 [0.34789535 0.19442797 0.17806861 ... 0.1385702  0.11007534 0.41939731]
 [0.37162815 0.25961438 0.09510807 ... 0.16916938 0.19597414 0.42165444]]


In [9]:
print(feature)

[[[0.25256538 0.17283795 0.24799582 ... 0.37072169 0.23484518 0.22808169]]

 [[0.13749049 0.21130778 0.1583412  ... 0.09388087 0.17400995 0.4289962 ]]

 [[0.15065896 0.2165108  0.28642376 ... 0.19594766 0.1089578  0.19535971]]

 ...

 [[0.19696707 0.25160873 0.23043812 ... 0.23055856 0.40360325 0.19155563]]

 [[0.18648838 0.08933366 0.19090642 ... 0.31531182 0.08991329 0.15113736]]

 [[0.20378755 0.16270082 0.0298022  ... 0.21203754 0.39301555 0.4954718 ]]]


In [31]:
similarity = np.dot(feature, feature.T)

In [32]:
print(similarity)

[[1.         0.75924657 0.14473213 ... 0.7827979  0.78997464 0.82221378]
 [0.75924657 1.         0.37213542 ... 0.82217253 0.83299421 0.8331656 ]
 [0.14473213 0.37213542 1.         ... 0.05212743 0.24047359 0.33027206]
 ...
 [0.7827979  0.82217253 0.05212743 ... 1.         0.82863578 0.71298453]
 [0.78997464 0.83299421 0.24047359 ... 0.82863578 1.         0.86095263]
 [0.82221378 0.8331656  0.33027206 ... 0.71298453 0.86095263 1.        ]]
